In [1]:
import pandas as pd
from datetime import datetime, timedelta
import numpy as np

sf = pd.read_csv("df_updated.csv")
sf


C:\Users\ppkk2\anaconda3\envs\python37\lib\site-packages\IPython\core\interactiveshell.py:3147: DtypeWarning: Columns (22) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Unnamed: 0,Unnamed: 0.1,DATE,NAME,SECTOR,VOLUME,PER,PBR,DIV,DPS,...,week2_기사_키워드평균,week2_기사_긍부정평균,week2_기사_count,week3_기사_키워드평균,week3_기사_긍부정평균,week3_기사_count,week4_기사_키워드평균,week4_기사_긍부정평균,week4_기사_count,month_기사_count
0,0,0,2018-01-02,현대건설,1,649218,8.44,0.65,1.35,500,...,5.000000,0.250000,14.0,15.714286,0.005952,23.0,1.714286,-0.333333,12.0,70.0
1,1,1,2018-01-03,현대건설,1,482518,8.24,0.64,1.38,500,...,3.571429,0.107143,10.0,12.285714,0.053571,21.0,5.428571,-0.309524,16.0,72.0
2,2,2,2018-01-04,현대건설,1,1459930,8.62,0.67,1.32,500,...,3.142857,0.089286,14.0,8.857143,0.107143,17.0,8.571429,-0.208333,18.0,73.0
3,3,3,2018-01-05,현대건설,1,655891,8.64,0.67,1.31,500,...,4.142857,0.128247,25.0,6.285714,0.250000,16.0,13.285714,-0.208333,17.0,74.0
4,4,4,2018-01-08,현대건설,1,577188,8.71,0.67,1.30,500,...,3.000000,-0.014610,21.0,6.000000,0.392857,16.0,14.714286,-0.279762,22.0,82.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39261,39261,39261,2021-02-22,유성기업,8,86083,4.74,0.30,6.22,200,...,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0
39262,39262,39262,2021-02-23,유성기업,8,121527,4.65,0.30,6.35,200,...,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0
39263,39263,39263,2021-02-24,유성기업,8,178389,4.45,0.28,6.63,200,...,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0
39264,39264,39264,2021-02-25,유성기업,8,98385,4.54,0.29,6.49,200,...,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0


In [2]:
#일부 종목 빼기
#sf = sf[~sf.NAME.isin(["셀트리온"])]
sf.NAME.unique()

array(['현대건설', 'LG화학', 'POSCO', '삼성SDI', '한국조선해양', 'S-Oil', '롯데케미칼',
       '롯데쇼핑', '한국가스공사', 'GS건설', 'SK이노베이션', 'NAVER', '삼성바이오로직스', '대한항공',
       '현대제철', '삼성전자', '호텔신라', '고려아연', 'LG이노텍', '한국전력', 'KT', '한전KPS',
       '현대글로비스', 'CJ제일제당', '카카오', '삼부토건', '진양화학', '조선선재', '하이스틸', '삼화전기',
       'KH 필룩스', '대우조선해양', '극동유화', '세이브존I&C', '서울가스', '미창석유', '대성산업',
       '한미약품', '동화약품', '유니퀘스트', '이월드', '삼아알미늄', '조선내화', '일진머티리얼즈', '써니전자',
       '세방', '현대위아', '계양전기', '태영건설', '롯데정밀화학', '광주신세계', '유성기업'],
      dtype=object)

In [4]:
set(os.listdir("종토방")).intersection(set(os.listdir("종토방_TEST")))

set()

In [5]:
set(sf.NAME.unique()) - set(map(lambda x: x[0:x.index('.')], os.listdir("종토방_긍부정평가결과")))

set()

In [6]:
dataframes=dict()
for filename in os.listdir("종토방_긍부정평가결과"):
    if filename.endswith(".csv"):
        stockname = filename.split(".")[0]
        try:
            dataframes[stockname] = pd.read_csv(f"종토방_긍부정평가결과/{filename}")
        except UnicodeDecodeError:
            dataframes[stockname] = pd.read_csv(f"종토방_긍부정평가결과/{filename}", encoding="cp949")

In [7]:
def bbs_sentiment(row):
    row = row.copy()
    criteria = pd.to_datetime(row["DATE"])
    stock_name = row["NAME"]

    try:

        df = dataframes[stock_name]
        df["date"] = pd.to_datetime(df["date"])
        for i in range(1,5):
            days_before = i*7
            temp_df = df[(( criteria - timedelta(days=days_before) ) <= df["date"]) & (df["date"] <= ( criteria - timedelta(days=days_before-6) ) )]
            #print(min(temp_df.date), max(temp_df.date), len(temp_df), temp_df.label.mean())
            row[f"week{i}_종토방_글수"] = len(temp_df)
            row[f"week{i}_종토방_긍부정평균"] = temp_df.label.mean()

    except KeyError:
        print("KeyError:",stock_name)

    except Exception as e:
        print(type(e), e, row)

    return row

In [8]:
new = sf.apply(bbs_sentiment, axis=1)
new

,Unnamed: 0,Unnamed: 0.1,DATE,NAME,SECTOR,VOLUME,PER,PBR,DIV,DPS,...,week4_기사_count,month_기사_count,week1_종토방_글수,week1_종토방_긍부정평균,week2_종토방_글수,week2_종토방_긍부정평균,week3_종토방_글수,week3_종토방_긍부정평균,week4_종토방_글수,week4_종토방_긍부정평균
0,0,0,2018-01-02,현대건설,1,649218,8.44,0.65,1.35,500,...,12.0,70.0,37,0.489850,51,0.542892,102,0.429942,65,0.487735
1,1,1,2018-01-03,현대건설,1,482518,8.24,0.64,1.38,500,...,16.0,72.0,32,0.502718,57,0.530411,99,0.447602,61,0.448210
2,2,2,2018-01-04,현대건설,1,1459930,8.62,0.67,1.32,500,...,18.0,73.0,33,0.536455,61,0.505641,87,0.435782,70,0.473345
3,3,3,2018-01-05,현대건설,1,655891,8.64,0.67,1.31,500,...,17.0,74.0,55,0.535718,61,0.511538,49,0.528806,104,0.429063
4,4,4,2018-01-08,현대건설,1,577188,8.71,0.67,1.30,500,...,22.0,82.0,78,0.529658,38,0.532232,54,0.539484,101,0.415629
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39261,39261,39261,2021-02-22,유성기업,8,86083,4.74,0.30,6.22,200,...,0.0,0.0,24,0.293066,17,0.524857,116,0.490507,18,0.421042
39262,39262,39262,2021-02-23,유성기업,8,121527,4.65,0.30,6.35,200,...,0.0,0.0,22,0.302272,15,0.457533,120,0.491911,17,0.462671
39263,39263,39263,2021-02-24,유성기업,8,178389,4.45,0.28,6.63,200,...,0.0,0.0,21,0.313428,19,0.372165,122,0.493149,13,0.554237
39264,39264,39264,2021-02-25,유성기업,8,98385,4.54,0.29,6.49,200,...,0.0,0.0,24,0.274343,16,0.362671,44,0.488564,92,0.501156


In [9]:
new["week1_종토방_긍부정평균"] = new["week1_종토방_긍부정평균"].fillna(0.5)
new["week2_종토방_긍부정평균"] = new["week2_종토방_긍부정평균"].fillna(0.5)
new["week3_종토방_긍부정평균"] = new["week3_종토방_긍부정평균"].fillna(0.5)
new["week4_종토방_긍부정평균"] = new["week4_종토방_긍부정평균"].fillna(0.5)

In [10]:
new["4주간_종토방_긍부정변동"]=abs(new["week4_종토방_긍부정평균"] - new["week3_종토방_긍부정평균"]) + abs(new["week3_종토방_긍부정평균"] - new["week2_종토방_긍부정평균"]) + abs(new["week2_종토방_긍부정평균"] - new["week1_종토방_긍부정평균"])
new["4주간_종토방_글수변동"]=abs(new["week4_종토방_글수"] - new["week3_종토방_글수"]) + abs(new["week3_종토방_글수"] - new["week2_종토방_글수"]) + abs(new["week2_종토방_글수"] - new["week1_종토방_글수"])

In [20]:
new2 = new.drop(["week1_종토방_글수","week1_종토방_긍부정평균","week2_종토방_글수","week2_종토방_긍부정평균","week3_종토방_글수","week3_종토방_긍부정평균","week4_종토방_글수", "week4_종토방_긍부정평균"], axis=1)

In [11]:
new.to_csv("df_updated_.csv")